## **Seperating Hospitalizations Per Patient**

We want to add an identifier for each hospitalization per patient to seperate the data for each hospitalization.
We will use the StartTime of the concepts "Hospitaliztion In" and "Hospitaliztion Out" to identify each hospitalization of a patient and all the records during this hospitalization.

1. Number the "Hospitaliztion In" and "Hospitaliztion Out" concepts of each patient.
- Added a column NumberOfHospitaliztion

In [ ]:
WITH MispurIshpuzIN AS (
    SELECT
        PatientID,
        ConceptName,
        StartTime,
        EndTime,
        value,
        ROW_NUMBER() OVER (PARTITION BY PatientID ORDER BY StartTime) AS NumberOfHospitaliztion
    FROM DiabeticResearch_Rambam.dbo.InputPatientsData
    WHERE ConceptName = 'Hospitaliztion In'
),
MispurIshpuzOUT AS (
    SELECT
        PatientID,
        ConceptName,
        StartTime,
        EndTime,
        value,
        ROW_NUMBER() OVER (PARTITION BY PatientID ORDER BY StartTime) AS NumberOfHospitaliztion
    FROM DiabeticResearch_Rambam.dbo.InputPatientsData
    WHERE ConceptName = 'Hospitaliztion out'
),


SyntaxError: invalid syntax (<ipython-input-1-8fa992bf4151>, line 1)

2. Adding the hospitalization number for all concepts between "Hospitaliztion In" and "Hospitaliztion Out".

In [ ]:
MispurIshpuzTotal AS (
    SELECT
        hb.PatientID,
        b.ConceptName,
        b.StartTime,
        b.EndTime,
        b.value,
        hb.NumberOfHospitaliztion
    FROM MispurIshpuzIN hb
    JOIN MispurIshpuzOUT ho
        ON hb.PatientID = ho.PatientID
        AND hb.NumberOfHospitaliztion = ho.NumberOfHospitaliztion
    JOIN DiabeticResearch_Rambam.dbo.InputPatientsData b
        ON b.PatientID = hb.PatientID
        AND b.StartTime >= hb.StartTime
        AND b.StartTime <= ho.StartTime
),


3. Removing the NumberOfHospitaliztion and concatinating the hospitalization number to the PatientID.

In [ ]:
MispurIshpuzTotal_ChangeIDs AS (
    SELECT
        CAST(PatientID AS NVARCHAR(500)) + '_' + CAST(NumberOfHospitaliztion AS NVARCHAR(500)) AS PatientID,
        ConceptName,
        StartTime,
        EndTime,
        value
    FROM MispurIshpuzTotal
),


# **Data retrieval for the hypoglycemia target group**

1. Search for a diagnosis of hypoglycemia all patients

In [ ]:
FirstGlucoseEvent AS (
    SELECT
        PatientID,
        MIN(StartTime) AS firstDate
    FROM MispurIshpuzTotal_ChangeIDs
    WHERE (conceptname LIKE '%Glucose (glucometer) - whole blood%'
           OR ConceptName LIKE '%Glucose-BL%'
           OR ConceptName LIKE '%Glucose-Body Fluids%')
      AND TRY_CAST(value AS FLOAT) <= 70
    GROUP BY PatientID
)


2. Retrieving all patients data up until the hypogycemia diagnosis

In [ ]:
SELECT
    MispurIshpuzTotal_ChangeIDs.*
FROM MispurIshpuzTotal_ChangeIDs
LEFT JOIN FirstGlucoseEvent fge
    ON MispurIshpuzTotal_ChangeIDs.PatientID = CAST(fge.PatientID AS NVARCHAR(500))
WHERE ((fge.firstDate IS NOT NULL AND MispurIshpuzTotal_ChangeIDs.StartTime <= fge.firstDate)
       OR (fge.firstDate IS NULL))
ORDER BY MispurIshpuzTotal_ChangeIDs.PatientID, MispurIshpuzTotal_ChangeIDs.StartTime;

In [ ]:
import platform
print(platform.architecture())


('64bit', 'ELF')


In [ ]:
import pyodbc
print(pyodbc.drivers())

import pandas as pd
import platform
print(platform.architecture())

# פרטי החיבור
server = '11.0.2100.60'
database = 'DiabeticResearch_Rambam'
username = 'visitors'
password = 'visitors'
driver = 'ODBC Driver 17 for SQL Server'

# מחרוזת החיבור עם הנהג המוגדר
connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=YES"

print(connection_string)

# חיבור לבסיס הנתונים
try:
    conn = pyodbc.connect(connection_string)
    print("Connection successful!")

    # ביצוע שאילתה
    query = "SELECT TOP 200 conceptname FROM InputPatientsData"
    df = pd.read_sql(query, conn)

    # הצגת התוצאות
    print(df)

    # סגירת החיבור
    conn.close()
except Exception as e:
    print(f"Error: {e}")


[]
('64bit', 'ELF')
DRIVER={ODBC Driver 17 for SQL Server};SERVER=11.0.2100.60;DATABASE=DiabeticResearch_Rambam;UID=visitors;PWD=visitors;TrustServerCertificate=YES
Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 17 for SQL Server' : file not found (0) (SQLDriverConnect)")


## הכנסת קוד לטבלה לפני סינון לפי קונספטים ודופק וקבוצת מטרה


In [ ]:
WITH MispurIshpuzIN AS (
    SELECT
        PatientID,
        ConceptName,
        StartTime,
        EndTime,
        value,
        ROW_NUMBER() OVER (PARTITION BY PatientID ORDER BY StartTime) AS NumberOfHospitaliztion
    FROM DiabeticResearch_Rambam.dbo.InputPatientsData
    WHERE ConceptName = 'Hospitaliztion In'
),
MispurIshpuzOUT AS (
    SELECT
        PatientID,
        ConceptName,
        StartTime,
        EndTime,
        value,
        ROW_NUMBER() OVER (PARTITION BY PatientID ORDER BY StartTime) AS NumberOfHospitaliztion
    FROM DiabeticResearch_Rambam.dbo.InputPatientsData
    WHERE ConceptName = 'Hospitaliztion out'
),
MispurIshpuzTotal AS (
    SELECT
        hb.PatientID,
        b.ConceptName,
        b.StartTime,
        b.EndTime,
        b.value,
        hb.NumberOfHospitaliztion
    FROM MispurIshpuzIN hb
    JOIN MispurIshpuzOUT ho
        ON hb.PatientID = ho.PatientID
        AND hb.NumberOfHospitaliztion = ho.NumberOfHospitaliztion
    JOIN DiabeticResearch_Rambam.dbo.InputPatientsData b
        ON b.PatientID = hb.PatientID
        AND b.StartTime >= hb.StartTime
        AND b.StartTime <= ho.StartTime
),
HospitalizationCTE AS (
    SELECT
        cast(CAST(PatientID AS NVARCHAR(500)) + CAST(NumberOfHospitaliztion AS NVARCHAR(500)) as int) AS PatientID,
        ConceptName,
        StartTime,
        EndTime,
        value
    FROM MispurIshpuzTotal
    WHERE ConceptName IN ('Hospitaliztion In', 'Hospitaliztion Out')
),
TimeDifferenceCTE AS (
    SELECT
        h_in.PatientID,
        DATEDIFF(HOUR, h_in.StartTime, h_out.EndTime) AS HoursDifference
    FROM HospitalizationCTE h_in
    JOIN HospitalizationCTE h_out
        ON h_in.PatientID = h_out.PatientID
        AND h_in.ConceptName = 'Hospitaliztion In'
        AND h_out.ConceptName = 'Hospitaliztion Out'
        AND h_out.StartTime > h_in.StartTime
),
FilteredPatients AS (
    SELECT DISTINCT
        PatientID
    FROM TimeDifferenceCTE
    WHERE HoursDifference >= 48
)
insert into Rambam_Hypoglycemia_Students.dbo.InputPatientsData(PatientID, ConceptName, StartTime, EndTime, Value)
SELECT
    cast(CAST(PatientID AS NVARCHAR(500)) + CAST(NumberOfHospitaliztion AS NVARCHAR(500)) as int) AS PatientID,
    ConceptName,
    StartTime,
    EndTime,
    value
FROM MispurIshpuzTotal
WHERE cast(CAST(PatientID AS NVARCHAR(500)) + CAST(NumberOfHospitaliztion AS NVARCHAR(500)) as int) IN (SELECT PatientID FROM FilteredPatients) and len(conceptname) <= 50
ORDER BY PatientID, StartTime;

## סינון אשפוזי יום:
מאושפזים שלא היה להם בדיקות לחץ דם, דופק ורמת סוכר בין השעות 18:00 עד 7:00 למחרת

In [ ]:
SELECT distinct PatientID FROM Rambam_Hypoglycemia_Students.dbo.InputPatientsData WHERE PatientID in(
SELECT  DISTINCT PatientID
    FROM Rambam_Hypoglycemia_Students.dbo.InputPatientsData
    WHERE
        (
            -- Filter rows where the StartTime or EndTime fall within the required time range
            CAST(StartTime AS TIME) >= '18:00:00'
            OR CAST(EndTime AS TIME) <= '07:00:00'
        )
        AND
        -- Filter rows where the ConceptName matches the desired patterns
        (
            ConceptName LIKE '%דיאסטולי%'
            OR ConceptName LIKE '%סיסטולי%'
            OR ConceptName LIKE '%glucometer%'
            OR ConceptName LIKE '%דופק%'
        )
		)

## Changing all types of Glucose concepts to 'Glucose'

In [ ]:
UPDATE [Rambam_Hypoglycemia_Students].[dbo].[InputPatientsData]
SET ConceptName = 'Glucose'
WHERE ConceptName IN ('Glucose (glucometer) - whole blood', 'Glucose-BL', 'Glucose-BL (POC)')

## Changing all antibiotics concepts to 'Antibiotics'

In [1]:

UPDATE [Rambam_Hypoglycemia_Students].[dbo].[InputPatientsData]
SET ConceptName = 'Antibiotics'
WHERE ConceptName IN (
    'Cefamezin',
    'Ceftazidime',
    'Diflucan',
    'Flagyl',
    'Metronidazole',
    'Noxafil',
    'Orbenil',
    'Pipril',
    'Resprim forte',
    'Tygacil',
    'Vancomycin',
    'Chloramphenicol',
    'Cidofovir',
    'Doxylin',
    'Epivir',
    'Foscavir',
    'Meronem',
    'Myambutol',
    'Penicillin',
    'Rebetol',
    'Septrin',
    'Cefovit',
    'Ciprogis',
    'Fluconazole',
    'Nystatin',
    'Picillin',
    'Sporanox',
    'Valganciclovir',
    'Vfend',
    'Amikin',
    'Amphotericin',
    'Gentamicin',
    'Klacid',
    'Linezolid',
    'Neomycin',
    'Oralten',
    'Rocephin',
    'Zeffix',
    'Albendazole',
    'Ceforal',
    'Ciproxin',
    'Erythromycin',
    'Hepsera',
    'Imipenem',
    'Karin',
    'Levofloxacin',
    'Moxyvit',
    'Resprim',
    'Rulid',
    'Sulfatrim',
    'Ancotil',
    'Clindamycin',
    'Doxy 100',
    'Eraxis',
    'Ertapenem',
    'Ethambutol',
    'Penibrin',
    'Rafapen mega',
    'Tamiflu',
    'Zinacef',
    'Augmentin',
    'Caspofungin',
    'Coliracin',
    'Moxypen',
    'Tazocin',
    'Viread',
    'Voriconazole',
    'Ziagen',
    'Amikacin',
    'Ampicillin',
    'Cefuroxime -teva',
    'Claforan',
    'Doxibiotic',
    'Erythrocin',
    'Lexiva',
    'Minocin',
    'Rafapen v.k.',
    'Amoxi',
    'Azenil',
    'Dalacin c',
    'Macrodantin',
    'Meropenem',
    'Tobi',
    'Valcyte',
    'Zeto',
    'Ambisome',
    'Diseptyl',
    'Fortum',
    'Invanz',
    'Keflex',
    'Minoclin',
    'Minocycline',
    'Pyrazinamide',
    'Ribavirin',
    'Rifampin',
    'Tarivid',
    'Tavanic',
    'Trican',
    'Unasyn',
    'Vanco-teva',
    'Zinnat',
    'Zovirax',
	'Chloramphenicol')






SyntaxError: invalid non-printable character U+00A0 (<ipython-input-1-942cdaf21325>, line 105)

## Changing 'דופק' to 'Heart Rate'

In [ ]:
UPDATE [Rambam_Hypoglycemia_Students].[dbo].[InputPatientsData]
SET ConceptName = 'Heart Rate'
WHERE   (ConceptName LIKE N'%דופק%')